<input data 경로 확인>
- 구글드라이브에 저장한 이미지 데이터를 사용하기 위해 구글드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<image data set 추가하는 과정> (**실행할 필요 X** > 다운받아서 DATA SET 폴더에 이미 합침)
- helmet, hat 사진 각 500장식 다운

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

In [ ]:
!pip3 install -r /content/OIDv4_ToolKit/requirements.txt --no-deps 
#에러 안뜨면 --no -deps 없애도 무방

In [ ]:
!python /content/OIDv4_ToolKit/main.py downloader --classes Hat --type_csv test --limit 500

In [ ]:
!python /content/OIDv4_ToolKit/main.py downloader --classes Helmet --type_csv test --limit 500

<데이터 Split>
- 모은 데이터셋을 train, valid, test 데이터로 0.8/0.1/0.1 비율로 나눔

In [1]:
#개인별 경로 설정 필요!
data_path = 'dataset/yolo_cropped_image_data/croped'
destination_path = 'dataset/yolo_cropped_image_data_splitted'

In [2]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
import splitfolders
folder_location = data_path
categories = ["helmet", "no helmet"]

for index, categories in enumerate(categories):
  splitfolders.ratio(folder_location, output=destination_path, seed=77, ratio=(0.8, 0.2))

Copying files: 1258 files [00:22, 56.14 files/s]
Copying files: 1258 files [00:04, 271.09 files/s]


<이미지 파일명 변경> (실행할 필요X, 필수적X > 이미 파일명 전부 변경하여 DATA SET에 반영함)
- 데이터 수 확인 및 데이터 상황 점검을 용이하게 하기 위해 이미지마다 번호 부여

In [ ]:
path1 = "/content/drive/MyDrive/프로젝트/image data/test/HELMET"
path2 = "/content/drive/MyDrive/프로젝트/image data/test/NO HELMET"
path3 = "/content/drive/MyDrive/프로젝트/image data/train/HELMET"
path4 = "/content/drive/MyDrive/프로젝트/image data/train/NO HELMET"
path5 = "/content/drive/MyDrive/프로젝트/image data/val/HELMET"
path6 = "/content/drive/MyDrive/프로젝트/image data/val/NO HELMET"

In [ ]:
#이미지파일 이름 일괄변경
import os
folder_path=path6 #path1~path6까지 전부 한번씩 실행해야함
folderlist=os.listdir(folder_path)

i=1

for name in folderlist:
    src = os.path.join(folder_path, name)
    dst = 'validNoHelmet' + str(i) + '.jpg' #디렉토리에 따라 test/train/valid + Helmet/NoHelmet으로 바꾸기
    dst = os.path.join(folder_path, dst)
    os.rename(src,dst)
    i += 1

<이미지 전처리>
- ImageDateGenerator를 사용하여 train data를 여러 방법으로 증강시킴
- flow_from_directory를 사용하여 train, valid, test data를 224*224 사이즈로 resize하고 numpy array iterator로 바꿈

In [6]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

In [4]:
#개인에 맞게 path 설정 필요!
train_path = 'dataset/yolo_cropped_image_data_splitted/train'
val_path = 'dataset/yolo_cropped_image_data_splitted/val'

In [7]:
#Creating generator for Training DataSet
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.1, #이미지 기울기
        zoom_range=0.1, #이미지 일부 확대
        horizontal_flip=True, #이미지 가로 뒤집기
        rescale=1/255., #값을 0과 1사이로 변경
        rotation_range = 45.0, #이미지 회전값
        width_shift_range=0.4, #좌우이동
        fill_mode='nearest' #input 경계의 바깥공간을 채우는 방법
        )
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224), #이미지 사이즈
        batch_size=64, #배치 사이즈
        class_mode='categorical' #Y값 변화방법
        )

#Creating generator for Validation DataSet
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1/255.)
val_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 1005 images belonging to 3 classes.
Found 253 images belonging to 3 classes.


<1-1. keras.VGG16 활용하기>
- 2번 코드와 분리하여 실행할 것

In [9]:
#instantiate a base model with pre-trained weigts.
base_model=keras.applications.VGG16(
    include_top=False,
    weights="imagenet", #pretrained data
    input_shape=(224,224,3))

In [10]:
#freeze the base model
base_model.trainable = False

In [11]:
#Create new model on top
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(4096,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(2,activation='softmax',kernel_initializer='glorot_normal'))

In [ ]:
#earlystopping 없는 버전
#Train the model on new data.
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,epochs=50,validation_data=val_generator,workers=10,use_multiprocessing=True)

Epoch 1/50


Exception in thread Thread-38:
Traceback (most recent call last):
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\site-packages\keras\utils\data_utils.py", line 759, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\site-packages\keras\utils\data_utils.py", line 736, in pool_fn
    pool = get_pool_class(True)(
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\pool.py", line 303, in _repopulate_po

In [ ]:
#earlystopping 있는 버전
#Train the model on new data.
callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)

model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
history=model.fit(train_generator,epochs=50,validation_data=val_generator,workers=10,use_multiprocessing=True, callbacks= [callback])

Epoch 1/50


Exception in thread Thread-30:
Traceback (most recent call last):
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\site-packages\keras\utils\data_utils.py", line 759, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\site-packages\keras\utils\data_utils.py", line 736, in pool_fn
    pool = get_pool_class(True)(
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\Users\mjluc\anaconda3\envs\pyln\lib\multiprocessing\pool.py", line 303, in _repopulate_po

In [ ]:
#모델의 레이어 구성 확인
model.summary()

<1-2. 결과 시각화>

In [ ]:
#Some visualizations
import matplotlib.pyplot as plt
#Loss
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()
#Accuracy
plt.plot(history.history['accuracy'],label='acc')
plt.plot(history.history['val_accuracy'],label='val_acc')
plt.legend()
plt.show()

<2. VGG16 layer 쌓아 구현>
- 1번 코드 실행한 경우 실행하지 말아야 함

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Activation

In [ ]:
base_model = Sequential()
base_model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
base_model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
base_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
base_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
base_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
base_model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
base_model.summary()

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=4096,activation="relu", kernel_initializer='he_normal'))
model.add(Dropout(0.35)) #overfitting 방지
model.add(Dense(units=4096,activation="relu", kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(units=2, activation="softmax",kernel_initializer='glorot_normal'))

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#Train the model on new data.
checkpoint = ModelCheckpoint(
    filepath = "vgg16_1.h5", 
    monitor='loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', 
    save_period=1)
early = EarlyStopping(
    monitor='loss', 
    min_delta=0, 
    patience=10, 
    verbose=1, 
    mode='auto')

model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
history=model.fit(
    x=train_generator,
    y=None,
    batch_size=None,
    epochs=20,
    verbose='auto',
    callbacks=[early, checkpoint],
    validation_split=0.0,
    validation_data=val_generator,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=10,
    use_multiprocessing=True,
)